<a href="https://colab.research.google.com/github/annisatwain/AnalisisSentimen/blob/main/fakereview_revisi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


install library yang dibutuhkan

In [2]:
!pip install nltk scikit-learn gensim pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 33.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [4]:
import nltk
import re
import gensim
from gensim import corpora
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models.ldamodel import LdaModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from gensim.models import Word2Vec
import numpy as np

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation
/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4

In [5]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

understanding data

In [7]:
data = pd.read_csv('/content/drive/MyDrive/Dataset /data_real.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


data yang ditampilkan berasal dari dataset sebelumnya dengan label komen negatif dan komen positif


In [8]:
data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0.1,Unnamed: 0,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko,Url,label
0,0,59,Pesanan gx sesuai dgn yg d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
1,1,70,Pengiriman dan respon super cepat. Tpi barang ...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
2,2,95,barang yg dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
3,3,117,"Php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,https://www.tokopedia.com/cahayabelawa/steples...,0
4,4,153,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,https://www.tokopedia.com/hmhhardware/staple-g...,0
...,...,...,...,...,...,...,...,...,...,...,...
1920,995,1017,Bagus dan memuaskan. Packing bagus cepat,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,https://www.tokopedia.com/indahjayatools/stapl...,1
1921,996,1018,Mantap pengiriman ok cepat recomended,5,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,https://www.tokopedia.com/indahjayatools/stapl...,1
1922,997,1019,Pengiriman cepat. Lumayan,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,https://www.tokopedia.com/indahjayatools/stapl...,1
1923,998,1020,belom dicoba semoga oke barangnya,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,https://www.tokopedia.com/indahjayatools/stapl...,1


In [9]:
data_new = data.copy()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


disini saya mencoba untuk membersihkan kolom yang tidak diperlukan

In [10]:
data_new.drop(['Unnamed: 0.1', 'Url', 'Unnamed: 0', 'label'], axis=1, inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
data_new

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko
0,Pesanan gx sesuai dgn yg d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393
1,Pengiriman dan respon super cepat. Tpi barang ...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393
2,barang yg dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393
3,"Php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298
4,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197
...,...,...,...,...,...,...,...
1920,Bagus dan memuaskan. Packing bagus cepat,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588
1921,Mantap pengiriman ok cepat recomended,5,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588
1922,Pengiriman cepat. Lumayan,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588
1923,belom dicoba semoga oke barangnya,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588


text prepocessing

In [12]:
reviews = data_new['Ulasan']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('indonesian'))
lemmatizer = WordNetLemmatizer()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
data_new['Ulasan_Bersih'] = reviews.apply(preprocess_text)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
data_new

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko,Ulasan_Bersih
0,Pesanan gx sesuai dgn yg d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,pesanan gx sesuai dgn yg d gambar
1,Pengiriman dan respon super cepat. Tpi barang ...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,pengiriman respon super cepat tpi barang sesua...
2,barang yg dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,barang yg dikirim sesuai pesanan
3,"Php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,php bilang isi diseskripsi pa dipesen berbeda ...
4,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,barang jelek copot tmpat taro isinya
...,...,...,...,...,...,...,...,...
1920,Bagus dan memuaskan. Packing bagus cepat,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,bagus memuaskan packing bagus cepat
1921,Mantap pengiriman ok cepat recomended,5,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,mantap pengiriman ok cepat recomended
1922,Pengiriman cepat. Lumayan,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,pengiriman cepat lumayan
1923,belom dicoba semoga oke barangnya,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,belom dicoba semoga oke barangnya


In [17]:
cleaned_reviews = data_new['Ulasan_Bersih']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


embedding

In [18]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(cleaned_reviews.tolist())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
data_new['embeddings'] = embeddings.tolist()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
tokenized_reviews = [review.split() for review in cleaned_reviews]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


pelabelan menggunakan LDA

In [21]:
dictionary = corpora.Dictionary(tokenized_reviews)
corpus = [dictionary.doc2bow(comment) for comment in tokenized_reviews]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=2, passes=10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
topics = lda_model.print_topics(num_words=5)
print("Topik yang ditemukan oleh LDA:")
for idx, topic in topics:
    print(f"Topik {idx}: {topic}")

lda_display = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)


def assign_topic(lda_model, comment):
    bow = dictionary.doc2bow(comment)
    topic_distribution = lda_model.get_document_topics(bow)
    return max(topic_distribution, key=lambda x: x[1])[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topik yang ditemukan oleh LDA:
Topik 0: 0.039*"barang" + 0.021*"bagus" + 0.017*"nya" + 0.015*"sesuai" + 0.013*"ga"
Topik 1: 0.046*"barang" + 0.037*"sesuai" + 0.021*"yg" + 0.018*"cepat" + 0.015*"terima"


In [24]:
data_new['topic'] = [assign_topic(lda_model, comment) for comment in tokenized_reviews]

data_new['komentar'] = data_new['topic'].apply(lambda x: 'bot' if x == 0 else 'human')

# Menampilkan dataset yang sudah dilabeli
print("\nData dengan label:")
print(data_new[['Ulasan_Bersih', 'komentar', 'topic']].head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Data dengan label:
                                       Ulasan_Bersih komentar  topic
0                  pesanan gx sesuai dgn yg d gambar    human      1
1  pengiriman respon super cepat tpi barang sesua...    human      1
2                   barang yg dikirim sesuai pesanan    human      1
3  php bilang isi diseskripsi pa dipesen berbeda ...      bot      0
4               barang jelek copot tmpat taro isinya    human      1


In [29]:
data_new

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko,Ulasan_Bersih,embeddings,topic,komentar
0,Pesanan gx sesuai dgn yg d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,pesanan gx sesuai dgn yg d gambar,"[-0.06962281465530396, 0.011958700604736805, -...",1,human
1,Pengiriman dan respon super cepat. Tpi barang ...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,pengiriman respon super cepat tpi barang sesua...,"[-0.0531228706240654, 0.10111096501350403, -0....",1,human
2,barang yg dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,barang yg dikirim sesuai pesanan,"[-0.042245276272296906, 0.12197387963533401, -...",1,human
3,"Php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,php bilang isi diseskripsi pa dipesen berbeda ...,"[-0.007697275839745998, 0.055699195712804794, ...",0,bot
4,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,barang jelek copot tmpat taro isinya,"[-0.08997959643602371, 0.18369898200035095, -0...",1,human
...,...,...,...,...,...,...,...,...,...,...,...
1920,Bagus dan memuaskan. Packing bagus cepat,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,bagus memuaskan packing bagus cepat,"[-0.035563722252845764, 0.0938347652554512, -0...",0,bot
1921,Mantap pengiriman ok cepat recomended,5,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,mantap pengiriman ok cepat recomended,"[-0.07521235197782516, 0.0185532346367836, -0....",0,bot
1922,Pengiriman cepat. Lumayan,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,pengiriman cepat lumayan,"[-0.015180530957877636, -0.004802112467586994,...",1,human
1923,belom dicoba semoga oke barangnya,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,belom dicoba semoga oke barangnya,"[-0.017231274396181107, 0.12312693893909454, -...",1,human


modelling

In [30]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Mengonversi embeddings yang berbentuk list ke dalam format array numpy (jika perlu)
X = np.array(data_new['embeddings'].tolist())  # Mengonversi kolom 'embeddings' menjadi array numpy
y = data_new['topic'].values  # Label biner (0 atau 1)

# Memisahkan data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Melatih model
rf_model.fit(X_train, y_train)

# Prediksi pada data uji
y_pred = rf_model.predict(X_test)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


evaluasi

In [31]:
# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7013
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.61      0.65       177
           1       0.70      0.78      0.74       208

    accuracy                           0.70       385
   macro avg       0.70      0.69      0.70       385
weighted avg       0.70      0.70      0.70       385



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
